In [1]:
import CloudEnvModule
import importlib
importlib.reload(CloudEnvModule) # update module in case of changes
from CloudEnvModule import CloudEnv # import custom environment
from CloudEnvModule import STOP, calculate_evenness, CpuMem
from gym.spaces import Box
import numpy as np

In [2]:
def rand(a, b):
    return np.random.randint(a, b)

class NewCloudEnv(CloudEnv):
    def __init__(self):
        super().__init__(n=30, m=3)
        self.timestamp = 0

    # def get_state(self):
    #     state = np.zeros(shape=(self.n, self.m, 5))
    #     for i in range(self.n):
    #         for j in range(self.m):
    #             state[i][j] = [
    #                 self.vms[i].cpu,
    #                 self.vms[i].mem,
    #                 self.servers[j].cpu,
    #                 self.servers[j].mem,
    #                 int(self.vms[i] <= self.servers[j])
    #             ]
    #     return state

    def get_reward(self):
        ans = 0
        ans -= self.timestamp
        for i in range(self.n):
            if self.vms[i].cpu == STOP:
                ans += 1
        ans *= 0.1
        ans += calculate_evenness(self.servers)
        return self.update_score(ans)

    def step(self, action):
        self.timestamp += 1
        return super().step(action)
    
    def reset(self):
        self.timestamp = 0
        self.vms = []
        self.servers = []
        for i in range(self.n):
            self.vms.append(CpuMem(rand(1, 4), rand(2, 8)))
        for i in range(self.m):
            self.servers.append(CpuMem(rand(30, 100), (60, 200)))
        self.was = 0
        return self.get_state()

env = NewCloudEnv()
# env = CloudEnv()

In [3]:
import DrawerModule
importlib.reload(DrawerModule) # update module in case of changes
from DrawerModule import Drawer

In [4]:
class GreedyModel:
    def __init__(self, env, get_order):
        self.order = get_order(env)
        self.timestamp = 0
    
    def predict(self, state):
        ans = self.order[0]
        if self.timestamp < len(self.order):
            ans = self.order[self.timestamp]
        self.timestamp += 1
        return ans, None

In [5]:
def getRandomVmServerOrder(env):
    done = False
    state = env.reset()
    def getCpuMem(vec):
        return CpuMem(vec[0], vec[1])
    order = []
    while not done:
        ans = [-1, -1]
        while ans == [-1, -1]:
            i = np.random.randint(env.n)
            j = np.random.randint(env.m)
            if getCpuMem(state['vms'][i]) <= getCpuMem(state['servers'][j]):
                ans = [i, j]
        state, reward, done, _ = env.step(ans)
        order.append(ans)
    return order

In [6]:
greedy_env = CloudEnv()
greedy_model = GreedyModel(greedy_env, getRandomVmServerOrder)

In [7]:
Drawer(greedy_env, greedy_model, rows=3, seconds_per_frame=0.1)